In [1]:
import flickrapi
from nltk import *
from nltk.corpus import stopwords
import ast

In [2]:
api_key = u'c0fbe33d0663c2a99981d03c9d21d9a1'
api_secret = u'8c76a2ad387c546e'

flickr = flickrapi.FlickrAPI(api_key, api_secret, format='json')

In [3]:
def get_continuous_chunks(text):
    chunked = ne_chunk(pos_tag(word_tokenize(text)))
    prev = None
    continuous_chunk = []
    current_chunk = []
    for i in chunked:
            if type(i) == Tree:
                current_chunk.append(i.label()[:3] + "-")
                current_chunk.append(" ".join([token for token, pos in i.leaves()]))
            elif current_chunk:
                named_entity = " ".join(current_chunk)
                if named_entity not in continuous_chunk:
                    continuous_chunk.append(named_entity)
                    current_chunk = []
            else:
                continue
    return continuous_chunk

In [4]:
def get_relation_tuples(sentences, image):
    relation_tuples = []
    for sentence in sentences:
        rel_tuple_single = []
        rel_tuple_single = get_continuous_chunks(sentence)
        #print(rel_tuple_single)

        if len(rel_tuple_single):
            tokens = word_tokenize(sentence)
            tagged = pos_tag(tokens)

        #extract named entities
        entities = chunk.ne_chunk(tagged)
        #print entities
        for entity in entities:

            if len(entity)>1 and (entity[1] == 'VB' or entity[1] == 'VBD' or entity[1] == 'VBN' or entity[1] == 'VM'):
                if entity[0] not in stopwords.words('english'):
                    rel_tuple_single.append('REL-'+entity[0])
        rel_tuple_single.append('IMG-'+image)

        #print entity
        if len(rel_tuple_single):
            relation_tuples.append(rel_tuple_single)

    with open("input.txt", "a") as myfile:
        myfile.write(str(relation_tuples)+'\n')

In [5]:
m = {
    "Dainik Vijay News" : '2339229@N21',
    "Crime Scene Photographers" : '49606819@N00'    
}

for m_ in m:
    photos = flickr.photos.search(group_id=m[m_], per_page='15')
    photos = photos.decode('utf-8')
    photos_dict = ast.literal_eval(photos)
    photos = photos_dict['photos']
    for photo in photos['photo']:
        title = photo['title']
        pic = 'https://farm'+str(photo['farm'])+'.staticflickr.com/'+photo['server']+'/'+photo['id']+'_'+photo['secret']+'.jpg'
        try:
            get_relation_tuples(sent_tokenize(title), pic)
        except:
            i = 1

In [ ]:
# https://farm1.staticflickr.com/4543/38537177461_ae0656d951_m.jpg